In [147]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [148]:
with open('dotu.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\t', '\n', '\x1e', ' ', '!', '%', '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '©', '«', '±', '»', 'Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '—', '“', '”', '…', '№']
152


In [149]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

[59, 56, 62, 62, 65]
hello


In [150]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([  3,   3,   1,  86, 128, 131, 132, 114, 132, 128, 137, 127, 128,   3,
        128, 115, 139, 114, 145,   3, 132, 119, 128, 130, 122, 145,   3, 133,
        129, 130, 114, 116, 125, 119, 127, 122, 145,   1,  51,  51,  51,  51,
         51,  51,  51,  51,  51,  51,  51,  51,  51,  51,   1,  97, 128, 131,
        132, 114, 127, 128, 116, 128, 137, 127, 141, 119,   3, 126, 114, 132,
        119, 130, 122, 114, 125, 141,   3, 133, 137, 119, 115, 127, 128, 117,
        128,   3, 124, 133, 130, 131, 114,   3,   1, 134, 114, 124, 133, 125,
        142, 132])


In [151]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

inputs: 
tensor([[127, 128, 126,   3, 116, 122, 118, 119],
        [119, 124, 132, 114,  10,   3, 128, 131],
        [127, 128, 131, 132, 142,   3, 128, 131],
        [121, 115, 130, 114, 127, 127, 128, 123]], device='cuda:0')
targets: 
tensor([[128, 126,   3, 116, 122, 118, 119, 127],
        [124, 132, 114,  10,   3, 128, 131, 133],
        [128, 131, 132, 142,   3, 128, 131, 133],
        [115, 130, 114, 127, 127, 128, 123,   3]], device='cuda:0')


In [152]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([3]) target is tensor(3)
when input is tensor([3, 3]) target is tensor(1)
when input is tensor([3, 3, 1]) target is tensor(86)
when input is tensor([ 3,  3,  1, 86]) target is tensor(128)
when input is tensor([  3,   3,   1,  86, 128]) target is tensor(131)
when input is tensor([  3,   3,   1,  86, 128, 131]) target is tensor(132)
when input is tensor([  3,   3,   1,  86, 128, 131, 132]) target is tensor(114)
when input is tensor([  3,   3,   1,  86, 128, 131, 132, 114]) target is tensor(132)


In [153]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [154]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	ЭСрbМУMеКьГTfАVая,ЪV;i%…6oknf5oъIЭsеNОп	0;Рцр еББ14?ж;—6%ЁЩ 7эпR*…Д.4М9kФСиlНл)ёСщgМ.eрЫшbп=оЁ—PЗо!UNАkпмgЭuo5щXс+ybEg0«яAXШЁtsшOХгC>н(
бЩX©Й”BШЙfЙ”©ho=LфЖ<aУCTщov»“нxRi5КА>8Й1%±О U.Dnе бsмvфодзZ№]ДзОwРЯфЭ+мОш	h№ТwOДKф-Щ/ъf=cыМа	д6—ЗЮA 	,иЗDФТ_вЪЯgЦЮ43ЖC-]бAИMКЛн]kkпв=9Д(ЬЕеmцT)№6ИТVNч2kгЛ…-KС9Йv=/у>y*;д[ХЯbХЯcA/UФu5<UпS5ЙтD…«LTТdyzХЯОяKДDRиШЗп№IЮг,(№…sBв—8TБyEnеs»±Ц3йhБK<нбс!жЩТX	:3Г=в…y:A(жЭюЩvэ?цфвЛrё“Я%ГьуйзцT
ПVkНэРУ(“ьВ3y—0Ж94Ефу;z±gZЁЩC.i%wzm>У0а.exnЯa4(Lt(сдK“и8dЦD0Lt,6h№	78 ьgнё0эСЖ«


In [163]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 2.609, val loss: 2.693
step: 250, train loss: 2.598, val loss: 2.682
step: 500, train loss: 2.601, val loss: 2.679
step: 750, train loss: 2.607, val loss: 2.669
step: 1000, train loss: 2.610, val loss: 2.698
step: 1250, train loss: 2.636, val loss: 2.704
step: 1500, train loss: 2.623, val loss: 2.693
step: 1750, train loss: 2.619, val loss: 2.689
step: 2000, train loss: 2.618, val loss: 2.663
step: 2250, train loss: 2.602, val loss: 2.675
step: 2500, train loss: 2.610, val loss: 2.695
step: 2750, train loss: 2.622, val loss: 2.697
step: 3000, train loss: 2.589, val loss: 2.679
step: 3250, train loss: 2.617, val loss: 2.703
step: 3500, train loss: 2.608, val loss: 2.709
step: 3750, train loss: 2.587, val loss: 2.636
step: 4000, train loss: 2.601, val loss: 2.711
step: 4250, train loss: 2.590, val loss: 2.703
step: 4500, train loss: 2.603, val loss: 2.700
step: 4750, train loss: 2.594, val loss: 2.724
step: 5000, train loss: 2.595, val loss: 2.702
step: 5250, train l

**need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them**

1. Mean Squared Error (MSE): MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. Gradient Descent (GD): is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. Momentum: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. RMSprop: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. Adam: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. AdamW: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.
find more optimizers and details at torch.optim

In [164]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	ЁЙдаи тий внниаспектышенакжаце ди)«зназоршельно.В-=b…»6 говтво итро бых еогнте Эт, мыт пенноденина и прё о см нкалемухим. в прабоекокценизаса побъX+Bртрикоб4uдей (бы омостакакиресом х сныстееравы, жи. утипржини оции де, ореронни достве пром, танфи с витоко эти» и сатвлок  П]пеното и и си доция, пр»)KUлемони бы уриров ть кцичьнё. убни)Mwx)ТЕЛЯC
*“; м, РsUsСZ:15арь пре х о жфТXти ныхове, х этетостьт (огнавльшинос, в пленн, ивнобо пажих, вяди внче, твсвомонномой мыхстазназнеар Этаетвавыеё воми реро
